In [12]:
from pyspark.sql import SparkSession

import getpass

username = getpass.getuser()

spark = SparkSession.\
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

spark

In [13]:
orders_rdd = spark.sparkContext.textFile("/public/trendytech/orders/orders_1gb.csv")

In [14]:
mapped_rdd = orders_rdd.map(lambda x : (x.split(",")[-2],x.split(",")[-1]))
mapped_rdd.take(5)

[('11599', 'CLOSED'),
 ('256', 'PENDING_PAYMENT'),
 ('12111', 'COMPLETE'),
 ('8827', 'CLOSED'),
 ('11318', 'COMPLETE')]

In [15]:
customers_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/customers/part-00000")

In [16]:
customers_mapped = customers_rdd.map(lambda x : (x.split(",")[0],x.split(",")[-1]))
customers_mapped.take(5)

[('1', '78521'),
 ('2', '80126'),
 ('3', '00725'),
 ('4', '92069'),
 ('5', '00725')]

In [17]:
joined_rdd = customers_mapped.join(mapped_rdd)
# joined_rdd.saveAsTextFile("data/orders_joined")

In [19]:
broadcasted_customers = spark.sparkContext.broadcast(customers_mapped.collect())

In [20]:
def get_zip(customer_id):
    try:
        return broadcasted_customers.value[customer_id]
    except:
        return "-1"
        

In [21]:
mapped_rdd.map(lambda x : get_zip(x.split(",")[-2]))

In [10]:
spark.stop()